In [115]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from netgen.csg import Pnt

make mesh

In [116]:
netgen_mesh = unit_square.GenerateMesh(maxh=1.0)
#netgen_dual = unit_square.GenerateMesh(maxh=1.0)
#netgen_primal = unit_square.GenerateMesh(maxh=1.0)
#mesh=Mesh(netgen_mesh) 

# elvol = Integrate(CoefficientFunction(1),mesh,element_wise=True)
# h = [(2*vol)**(1/2) for vol in elvol]
# print(min(h),max(h))


In [117]:
mesh = Mesh(netgen_mesh)
#mesh.Refine()
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [118]:
n_v = mesh.nv
old_f = mesh.nface
print(n_v)
mesh.ngmesh.SplitPowellSabin()
n_f = mesh.nface - old_f
print(n_f)


4
12


In [119]:

#help(mesh)
Draw(mesh)

# for f in mesh.faces:
#     print (f, f.vertices)

data_length = 0

for i in range(n_v):
    v = mesh.vertices.__getitem__(i)
    print(v, v.point)
    print(v.faces)
    data_length += len(v.faces)



WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

V0 (0.0, 0.0)
(F2, F3, F4, F5)
V1 (1.0, 0.0)
(F6, F7)
V2 (1.0, 1.0)
(F8, F9, F10, F11)
V3 (0.0, 1.0000000000000002)
(F12, F13)


In [120]:
netgen_primal = unit_square.GenerateMesh(maxh=1.0)
primal_mesh = Mesh(netgen_primal)
primal_space = H1(primal_mesh, order=1)
u_p = primal_space.TrialFunction()
v_p = primal_space.TestFunction()

m_p = BilinearForm(primal_space)
m_p += u_p * v_p *dx
m_p.Assemble()

dual_space = L2(mesh,order=0)
u_d = dual_space.TrialFunction()
v_d = dual_space.TestFunction()

#help(H1(mesh, order=1))

m_d = BilinearForm(dual_space) #trialspace=primal_space, testspace=dual_space)

m_d += u_d * v_d * dx

m_d.Assemble()

print(m_d.mat.ToDense())

#help(m.mat)

 0.0833333       0       0       0       0       0       0       0       0       0       0       0
       0 0.0833333       0       0       0       0       0       0       0       0       0       0
       0       0 0.0833333       0       0       0       0       0       0       0       0       0
       0       0       0 0.0833333       0       0       0       0       0       0       0       0
       0       0       0       0 0.0833333       0       0       0       0       0       0       0
       0       0       0       0       0 0.0833333       0       0       0       0       0       0
       0       0       0       0       0       0 0.0833333       0       0       0       0       0
       0       0       0       0       0       0       0 0.0833333       0       0       0       0
       0       0       0       0       0       0       0       0 0.0833333       0       0       0
       0       0       0       0       0       0       0       0       0 0.0833333       0       0
       0  

In [121]:
data = np.empty(data_length)
row_ind = np.empty(data_length, dtype=int)
col_ind = np.empty(data_length, dtype=int)
shape = (n_f,n_v)

j = 0

for i in range(n_v):
    v = mesh.vertices.__getitem__(i)
    #print(v.faces)
    coeff =  1/len(v.faces)
    for f in v.faces:
        #print(f)
        dofNr = dual_space.GetDofNrs(f)
        #print(dofNr)
        row_ind[j] = dofNr[0]
        col_ind[j] = i
        data[j] = coeff
        j += 1

# print(data)
# print(row_ind)
# print(col_ind)

# sci_mat = sp.sparse.csr_matrix((data, (row_ind, col_ind)),shape)
# print(sci_mat.todense())
# sci_coo = sci_mat.tocoo()

ng_mat = la.SparseMatrixdouble.CreateFromCOO(row_ind,col_ind,data, n_f,n_v)
print(ng_mat.ToDense())



    0.25       0       0       0
    0.25       0       0       0
       0     0.5       0       0
       0     0.5       0       0
       0       0    0.25       0
       0       0    0.25       0
    0.25       0       0       0
       0       0    0.25       0
       0       0    0.25       0
       0       0       0     0.5
       0       0       0     0.5
    0.25       0       0       0



In [122]:
print(m_p.mat.ToDense())

M = ng_mat.T @ m_d.mat @ ng_mat

print(M.ToDense())

 0.166667 0.0416667 0.0833333 0.0416667
 0.0416667 0.0833333 0.0416667       0
 0.0833333 0.0416667 0.166667 0.0416667
 0.0416667       0 0.0416667 0.0833333

 0.0208333       0       0       0
       0 0.0416667       0       0
       0       0 0.0208333       0
       0       0       0 0.0416667

